In [1]:
%pylab

Using matplotlib backend: WXAgg
Populating the interactive namespace from numpy and matplotlib


In [23]:
from skimage.draw import line, set_color

In [43]:
class Canvas(object):
    def __init__(self,ax):
        self.ax = ax

        # Set limits to unit square
        self.template = zeros((10, 10))
        
        self.ax.set_xlim((-1,10))
        self.ax.set_ylim((-1,10))

        # turn off axis
        self.ax.set_xticklabels([])
        self.ax.set_yticklabels([])

        # Create handle for a path of connected points
        self.path, = ax.plot([],[],'o-',lw=3)
        self.vert = []
        self.ax.set_title('LEFT: new point, MIDDLE: delete last point, RIGHT: close polygon')

        self.x = [] 
        self.y = []

        self.mouse_button = {1: self._add_point, 2: self._delete_point, 3: self._close_polygon}
        
    def set_location(self,event):
        if event.inaxes:
            self.x = event.xdata
            self.y = event.ydata
               
    def _add_point(self):
        self.vert.append((self.x,self.y))

    def _delete_point(self):
        if len(self.vert)>0:
            self.vert.pop()

    def _close_polygon(self):
        self.vert.append(self.vert[0])

    def update_path(self,event):

        # If the mouse pointer is not on the canvas, ignore buttons
        if not event.inaxes: return

        # Do whichever action correspond to the mouse button clicked
        self.mouse_button[event.button]()
        
        x = [self.vert[k][0] for k in range(len(self.vert))]
        y = [self.vert[k][1] for k in range(len(self.vert))]
        self.path.set_data(x,y)
        
        self.template=zeros((10,10))
        for i in range(0,10):
            for j in range(0, 10):
                if point_in_poly(i, j, zip(x,y)):
                    self.template[j,i]=1
        if len(self.vert) > 1:
            for a,b in zip(self.vert[:-1], self.vert[1:]):
                set_color(self.template, line(int(round(a[1])), int(round(a[0])), int(round(b[1])), int(round(b[0]))), 2)
        imshow(self.template, interpolation='nearest')
        
        plt.draw()
        
def point_in_poly(x,y,poly):

    n = len(poly)
    inside = False

    p1x,p1y = poly[0]
    for i in range(n+1):
        p2x,p2y = poly[i % n]
        if y > min(p1y,p2y):
            if y <= max(p1y,p2y):
                if x <= max(p1x,p2x):
                    if p1y != p2y:
                        xints = (y-p1y)*(p2x-p1x)/(p2y-p1y)+p1x
                    if p1x == p2x or x <= xints:
                        inside = not inside
        p1x,p1y = p2x,p2y

    return inside

In [44]:
fig = plt.figure(1,(8,8))
template=zeros((10,10))
ax = fig.add_subplot(111)
cnv = Canvas(ax)
plt.connect('button_press_event',cnv.update_path)
plt.connect('motion_notify_event',cnv.set_location)

imshow(template)
plt.show()

In [42]:
a=cnv.vert[0]
b=cnv.vert[1]

set_color(cnv.template, line(int(round(a[1])), int(round(a[0])), int(round(b[1])), int(round(b[0]))), 2)

In [41]:
round(a[0])

4.0

In [5]:
import numpy as np
import matplotlib.path as mpath
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt

Path = mpath.Path

fig, ax = plt.subplots()

pathdata = [
    (Path.MOVETO, (1.58, -2.57)),
    (Path.CURVE4, (0.35, -1.1)),
    (Path.CURVE4, (-1.75, 2.0)),
    (Path.CURVE4, (0.375, 2.0)),
    (Path.LINETO, (0.85, 1.15)),
    (Path.CURVE4, (2.2, 3.2)),
    (Path.CURVE4, (3, 0.05)),
    (Path.CURVE4, (2.0, -0.5)),
    (Path.CLOSEPOLY, (1.58, -2.57)),
    ]

codes, verts = zip(*pathdata)
path = mpath.Path(verts, codes)
patch = mpatches.PathPatch(path, facecolor='green', edgecolor='yellow', alpha=0.5)
ax.add_patch(patch)


class PathInteractor:
    """
    An path editor.

    Key-bindings

      't' toggle vertex markers on and off.  When vertex markers are on,
          you can move them, delete them


    """

    showverts = True
    epsilon = 5  # max pixel distance to count as a vertex hit

    def __init__(self, pathpatch):

        self.ax = pathpatch.axes
        canvas = self.ax.figure.canvas
        self.pathpatch = pathpatch
        self.pathpatch.set_animated(True)

        x, y = zip(*self.pathpatch.get_path().vertices)

        self.line, = ax.plot(x,y,marker='o', markerfacecolor='r', animated=True)

        self._ind = None # the active vert

        canvas.mpl_connect('draw_event', self.draw_callback)
        canvas.mpl_connect('button_press_event', self.button_press_callback)
        canvas.mpl_connect('key_press_event', self.key_press_callback)
        canvas.mpl_connect('button_release_event', self.button_release_callback)
        canvas.mpl_connect('motion_notify_event', self.motion_notify_callback)
        self.canvas = canvas


    def draw_callback(self, event):
        self.background = self.canvas.copy_from_bbox(self.ax.bbox)
        self.ax.draw_artist(self.pathpatch)
        self.ax.draw_artist(self.line)
        self.canvas.blit(self.ax.bbox)

    def pathpatch_changed(self, pathpatch):
        'this method is called whenever the pathpatchgon object is called'
        # only copy the artist props to the line (except visibility)
        vis = self.line.get_visible()
        plt.Artist.update_from(self.line, pathpatch)
        self.line.set_visible(vis)  # don't use the pathpatch visibility state


    def get_ind_under_point(self, event):
        'get the index of the vertex under point if within epsilon tolerance'

        # display coords
        xy = np.asarray(self.pathpatch.get_path().vertices)
        xyt = self.pathpatch.get_transform().transform(xy)
        xt, yt = xyt[:, 0], xyt[:, 1]
        d = np.sqrt((xt-event.x)**2 + (yt-event.y)**2)
        ind = d.argmin()

        if d[ind]>=self.epsilon:
            ind = None

        return ind

    def button_press_callback(self, event):
        'whenever a mouse button is pressed'
        if not self.showverts: return
        if event.inaxes==None: return
        if event.button != 1: return
        self._ind = self.get_ind_under_point(event)

    def button_release_callback(self, event):
        'whenever a mouse button is released'
        if not self.showverts: return
        if event.button != 1: return
        self._ind = None

    def key_press_callback(self, event):
        'whenever a key is pressed'
        if not event.inaxes: return
        if event.key=='t':
            self.showverts = not self.showverts
            self.line.set_visible(self.showverts)
            if not self.showverts: self._ind = None

        self.canvas.draw()

    def motion_notify_callback(self, event):
        'on mouse movement'
        if not self.showverts: return
        if self._ind is None: return
        if event.inaxes is None: return
        if event.button != 1: return
        x,y = event.xdata, event.ydata

        vertices = self.pathpatch.get_path().vertices

        vertices[self._ind] = x,y
        self.line.set_data(zip(*vertices))

        self.canvas.restore_region(self.background)
        self.ax.draw_artist(self.pathpatch)
        self.ax.draw_artist(self.line)
        self.canvas.blit(self.ax.bbox)


interactor = PathInteractor(patch)
ax.set_title('drag vertices to update path')
ax.set_xlim(-3,4)
ax.set_ylim(-3,4)

plt.show()